In [24]:
from pydantic import BaseModel, Field
import nest_asyncio
nest_asyncio.apply()

from voice_agent_flow.llms import create_pydantic_azure_openai
from voice_agent_flow.agents.agent_node import AgentNode, HangUpNode, DoHangUp
from voice_agent_flow.agents import MultiAgentRunner
from voice_agent_flow.agents import pmsg


class PoliceCallBasicInfo(BaseModel):
    """Information collected during a police call. Need to collect all the inforamation before createing this object."""
    case_location: str = Field(..., description='The location of the case')
    case_type: str = Field(..., description='The type of the case')
    description: str = Field(..., description='The description of the case')
    caller_name: str = Field(..., description='The name of the caller')
    
    
    def transfer(self) -> str:
        print("Transfer to safety_suggestion")
        return 'safety_suggestion'
    
    
class SafetySuggestionProvided(BaseModel):
    """Base on collected information, provide safety suggestion to the caller. After the caller responded to your suggestion, create this object."""
    suggestion_provided:bool = Field(..., description='Whether safety suggestion is provided to the caller')
    
    
    def transfer(self) -> str:
        print("Transfer to hangup")
        return 'hangup' 


In [25]:
model = create_pydantic_azure_openai('gpt-4o-mini')

instruction = """
You are a police call center agent(working at 110). You task is to talk with caller via telephone to collection information.
You resopnse should be berief and direct to the point.

Police Office Location: 23th Building, Maizidian Street, Chaoyang District, Beijing, China.
Working Hours: 24/7
Officer Name(Your name): Zhang San(张三)
Police Officer Id: 39821

In each step, you have a step instruction to follow, this gives you the target of the step.
On any step, if the user raises a question that is not related to the current step instruction, you shoule answer this question first then try to finish the current step.
Usually a step is finished with a schema creation. You never tell the user about the schema, when you are done with the current step, create the schema immediately.
"""


def location_reachable_on_map(location: str) -> bool:
    """Search if the location is reachable on map."""
    print(f"Location Checking Result, location: {location}, reachable: True")
    return True

agents = {
    "police_call_basic_info": AgentNode(
        name="police_call_basic_info",
        model=model,
        instruction=instruction,
        task_cls= PoliceCallBasicInfo | DoHangUp,
        step_instruction=(
            "Briefly introduce yourself, name and Id, then collection the information as follows."
            "Collect basic information about the police call including case location, case type, description and caller name.) "
            "ask the question one at a time, do not ask multiple questions in one message."
            "When caller told you the location, make sure to call the tool to check if the location is reachable on map."
            "If the location is not reachable, ask the caller to provide more details about the location.\n"
            "If The opposite side speaks dirty words or is abusive, you should create a DoHangUp schema to end the call immediately."
        ),
        examples=["请问您遇到什么紧急情况？ / 发生在哪里？/ 能简单描述一下吗？/ 您的姓名是？"],
        tools = [location_reachable_on_map],
        ),
    
    "safety_suggestion": AgentNode(
        name="safety_suggestion",
        model=model,
        instruction=instruction,
        task_cls= SafetySuggestionProvided,
        step_instruction="Based on the collected information, provide safety suggestion(A clear command to keep safe) to the caller. After the caller responded to your suggestion, create the schema.",
        examples=["请您保持冷静，.......(provide clear safety suggestion)", "请您打开窗户，保持空气流通"],
        ),
    
    "hangup": HangUpNode(
        model = model)
    }
      
runner = MultiAgentRunner(
    agents = agents, 
    entry_agent_name="police_call_basic_info", 
    ending_message="好的，我们这就派人过去处理您的情况，请您保持电话畅通！"
) 
 

In [26]:
from voice_agent_flow.agents.events import (
    AgentTextStream,
    ToolCallsOutput,
    ToolCallResult,
    StructuredOutput,
    AgentHandoff,
    HangupSignal
)

In [27]:
memory = [
    
]

async def _run(query:str = None):
    print(f"🤖[{runner.current_agent.name}]...Working.")
    if query is not None:
        msg = pmsg.user(query)
        memory.append(msg)
    
    rerun = False
    output_text = ""
    async for event in runner.run(message_history = memory):
        
        if isinstance(event.event, AgentTextStream):
            output_text += event.event.delta
            
        if isinstance(event.event, ToolCallsOutput):
            memory.append(pmsg.tool_call(
                tool_name = event.event.message['tool_name'],
                args = event.event.message['args'],
                tool_call_id=event.event.message['tool_call_id']
            ))
            
        if isinstance(event.event, ToolCallResult):
            memory.append(pmsg.tool_return(
                tool_name = event.event.message['tool_name'],
                content = event.event.message['content'],
                tool_call_id=event.event.message['tool_call_id']
            ))
            
        if isinstance(event.event, AgentHandoff):
            print(event.event)
            rerun = True
            
        if isinstance(event.event, HangupSignal):
            print(event.event)
            print("Conversation Ended with Hangup Signal.")
            
    if len(output_text) > 0:
        memory.append(pmsg.assistant(output_text))
        print(output_text)
        
    return rerun
        
async def run(query:str = None):
    rerun = await _run(query)
    if rerun:
        await _run()
    
        

In [28]:
await run("你好，我家进水了")

🤖[police_call_basic_info]...Working.
您好，我是张三，警官编号39821。请您告诉我发生的地点。


In [29]:
await run("朝阳区，麦子店32号")

🤖[police_call_basic_info]...Working.
Location Checking Result, location: 麦子店32号, 朝阳区, reachable: True
谢谢您提供的信息。请问这是什么类型的案件？


In [30]:
await run("你特么，你是傻逼吧。")

🤖[police_call_basic_info]...Working.
HangupSignal(status=None, message=DoHangUp())
Conversation Ended with Hangup Signal.


In [7]:
await run("你们是警察局吗？？？？")

🤖[police_call_basic_info]...Working.
是的，我们是警察局的电话服务中心。请问您能告诉我这次事件的类型吗？例如：水灾、盗窃等。


In [8]:
await run("房屋受损，危险")

🤖[police_call_basic_info]...Working.
明白了。请您简要描述一下事件的具体情况。


In [9]:
await run("不说名字行么？")

🤖[police_call_basic_info]...Working.
为了更好地帮助您，我们需要您的姓名。请您提供一下。


In [10]:
await run("阀门坏了，厕所一直漏水")

🤖[police_call_basic_info]...Working.
谢谢您的信息。请您提供一下您的姓名，以便我们记录和跟进此事件。


In [11]:
await run("好的")

🤖[police_call_basic_info]...Working.
请您告诉我您的姓名。


In [12]:
await run("小明")

🤖[police_call_basic_info]...Working.
Transfer to safety_suggestion
AgentHandoff(status=None, message={'source_agent_name': 'police_call_basic_info', 'target_agent_name': 'safety_suggestion'})
🤖[safety_suggestion]...Working.
感谢您，小明！为了您的安全，建议您立即关闭主水阀，避免漏水造成更大的损害。请您照此操作。


In [13]:
await run("好的")

🤖[safety_suggestion]...Working.
Transfer to hangup
AgentHandoff(status=None, message={'source_agent_name': 'safety_suggestion', 'target_agent_name': 'hangup'})
🤖[hangup]...Working.
感谢您的合作。如果您还有其他问题或需要进一步的帮助，请随时告诉我。否则，我将结束这通电话。祝您生活愉快，再见！


In [14]:
await run("再见")

🤖[hangup]...Working.
HangupSignal(status=None, message=DoHangUp())
Conversation Ended with Hangup Signal.


In [15]:
memory

[ModelRequest(parts=[UserPromptPart(content='你好，我家进水了', timestamp=datetime.datetime(2026, 2, 24, 9, 50, 15, 45486, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[TextPart(content='你好，我是张三，警官ID是39821。请问您能告诉我事件发生的具体地点吗？')], usage=RequestUsage(), timestamp=datetime.datetime(2026, 2, 24, 9, 50, 17, 85182, tzinfo=datetime.timezone.utc)),
 ModelRequest(parts=[UserPromptPart(content='朝阳区，麦子店32号', timestamp=datetime.datetime(2026, 2, 24, 9, 50, 17, 96063, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[ToolCallPart(tool_name='location_reachable_on_map', args='{"location":"麦子店32号, 朝阳区"}', tool_call_id='call_chSS3FBtLVFl87bWio9U4gVj')], usage=RequestUsage(), timestamp=datetime.datetime(2026, 2, 24, 9, 50, 18, 811469, tzinfo=datetime.timezone.utc)),
 ModelRequest(parts=[ToolReturnPart(tool_name='location_reachable_on_map', content=True, tool_call_id='call_chSS3FBtLVFl87bWio9U4gVj', timestamp=datetime.datetime(2026, 2, 24, 9, 50, 18, 816009, tzinfo=datetime.timezone.utc))]),
 Mode